In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [3]:
!wget -q -O - ipv4.icanhazip.com

34.31.248.173


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Project fashion Recommender/Final Model.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Project fashion Recommender/New Images.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [7]:
%%writefile app.py
import streamlit as st
import os
from PIL import Image
import numpy as np
import pickle
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm

feature_list = np.array(pickle.load(open('/content/feature_lists.pkl','rb')))
filenames = pickle.load(open('/content/filenames.pkl','rb'))

model = ResNet50(weights = 'imagenet',include_top = False,input_shape = (224,224,3))
model.trainable = False
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

def feature_extraction(img_path,model):
  img = image.load_img(img_path,target_size = (224,224))
  img_array = image.img_to_array(img)
  expanded_img_array = np.expand_dims(img_array,axis = 0)
  preprocessed_img = preprocess_input(expanded_img_array)
  result = model.predict(preprocessed_img).flatten()
  normalized_result = result / norm(result)
  return normalized_result

def recommend(features,feature_list):
  neighbors = NearestNeighbors(n_neighbors = 5,algorithm='brute',metric = 'euclidean')
  neighbors.fit(feature_list)
  distances,indices = neighbors.kneighbors([features])
  return indices


st.title("Fashion Recommender System")

def save_uploaded_file(uploaded_file):
  try:
    with open(os.path.join('/content/drive/MyDrive/Project fashion Recommender/Uploaded_images',uploaded_file.name),'wb') as f:
      f.write(uploaded_file.getbuffer())
    return 1
  except:
    return 0


uploaded_file = st.file_uploader("Choose an Image: ")
if uploaded_file is not None:
  if save_uploaded_file(uploaded_file):
    display_img = Image.open(uploaded_file)
    st.image(display_img)
    # feature_extraction
    features = feature_extraction(os.path.join('/content/drive/MyDrive/Project fashion Recommender/Uploaded_images',uploaded_file.name),model)
    # st.text(features)
    st.text("Here are the products you may also them...")
    # recommend
    indices = recommend(features,feature_list)
    # display
    col1,col2,col3,col4,col5 = st.columns(5)
    img1 = Image.open(filenames[indices[0][0]])
    col1.image(img1)
    img2 = Image.open(filenames[indices[0][1]])
    col2.image(img2)
    img3 = Image.open(filenames[indices[0][2]])
    col3.image(img3)
    img4 = Image.open(filenames[indices[0][3]])
    col4.image(img4)
    img5 = Image.open(filenames[indices[0][4]])
    col5.image(img5)
  else:
    st.header("Some error occured in file upload")

Overwriting app.py


In [8]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.31.248.173:8501

npx: installed 22 in 5.933s
your url is: https://mighty-otters-drive.loca.lt
2024-05-14 11:04:38.370610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 11:04:38.370670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 11:04:38.372251: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 11:04:38.382012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructi